### 1. Transform Data from Wikipedia page

In [1]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
import requests # to send requesto url
print('Libraries imported')

Libraries imported


In [2]:
#create url
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [3]:
#Request web server to connect
Toronto_url = requests.get(url)
Toronto_url

<Response [200]>

In [4]:
#read text from the url with read_html
Toronta_data = pd.read_html(Toronto_url.text)

In [5]:
#check how many tables are in the html
print('There are {} tables type of {}'.format(len(Toronta_data), type(Toronta_data)))

There are 3 tables type of <class 'list'>


In [6]:
# Take the table that with information for this project
Toronto_final = Toronta_data[0]
Toronto_final.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


#### Rename Postcode

In [7]:
Toronto_final.rename(columns={'Postcode':'Postal Code'}, inplace = True)
Toronto_final.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


#### Remove rows with Borough 'Not Assigned'

In [8]:
print('Size of Toronto_final before drop', Toronto_final.shape)

IndexNames =  Toronto_final[Toronto_final['Borough']=='Not assigned'].index
Toronto_final.drop(IndexNames,inplace = True)

print('Size of Toronto_final after drop', Toronto_final.shape)

Toronto_final.head()

Size of Toronto_final before drop (288, 3)
Size of Toronto_final after drop (211, 3)


,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


#### Check Neighbourhood if there are any 'Not Assinged' and replace with borough value

In [9]:
for n in Toronto_final[Toronto_final['Neighbourhood']=='Not assigned'].index:
    Toronto_final.loc[n].at['Neighbourhood'] = Toronto_final.loc[n].at['Borough']

IndexNames =  Toronto_final[Toronto_final['Neighbourhood']=='Not assigned'].index
IndexNames

Int64Index([], dtype='int64')

#### Group Neighbourhoods in same PostalCode/Borough

In [10]:
Toronto_df = Toronto_final.groupby(['Postal Code','Borough'], as_index=False, sort = False).agg(', '.join)
Toronto_df.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


In [11]:
print('Size of Toronto_df after groupby', Toronto_df.shape)

Size of Toronto_df after groupby (103, 3)


### End of data transform

### 2. get the latitude and the longitude coordinates of each neighborhood.

In [12]:
!wget -q -O 'Geospatial_data.csv' https://cocl.us/Geospatial_data
print('Data downloaded!')

Data downloaded!


In [13]:
Geospatial_df = pd.read_csv('Geospatial_data.csv')
Geospatial_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [14]:
print('size of Geospatial_df is', Geospatial_df.shape)
print('size of Toronto_df is', Toronto_df.shape)

size of Geospatial_df is (103, 3)
size of Toronto_df is (103, 3)


In [15]:
Toronto_df = pd.merge(Toronto_df, Geospatial_df, on='Postal Code',how='inner')
Toronto_df.head(11)

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937


In [16]:
Toronto_df.shape

(103, 5)

### End of latitude and the longitude